<a href="https://colab.research.google.com/github/ShotaroBaba/NLP_Practice/blob/NLP_smaller_dataset/NLPIMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Firstly, download text dataset.:
import os
import gensim
import requests 
from gensim.parsing.preprocessing import stem
from gensim.parsing.preprocessing import strip_punctuation

from gensim.corpora import Dictionary
urlToWiki = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
savedFileName = 'aclImdb_v1.tar.gz'
if not os.path.isfile(savedFileName):
  with open(savedFileName,'wb') as output:
    output.write(requests.get(urlToWiki).content)

In [0]:
import tarfile

# Extract tar.gz file
tarfile.open(savedFileName, "r:gz").extractall()

In [0]:
neg_test = [os.path.join(root,x) for root, _, files in os.walk("aclImdb/test/neg/", topdown=False) for x in files if x.endswith(".txt") ]
pos_test  = [os.path.join(root,x) for root, _, files in os.walk("aclImdb/test/pos/", topdown=False) for x in files if x.endswith(".txt") ]

In [53]:
print(neg_test[:10])

['aclImdb/test/neg/5408_1.txt', 'aclImdb/test/neg/10830_1.txt', 'aclImdb/test/neg/10039_1.txt', 'aclImdb/test/neg/1777_4.txt', 'aclImdb/test/neg/6750_4.txt', 'aclImdb/test/neg/2079_1.txt', 'aclImdb/test/neg/9_4.txt', 'aclImdb/test/neg/1449_4.txt', 'aclImdb/test/neg/7144_4.txt', 'aclImdb/test/neg/3739_2.txt']


In [0]:
neg_train = [os.path.join(root,x) for root, _, files in os.walk("aclImdb/train/neg/", topdown=False) for x in files if x.endswith(".txt")]
pos_train  =[os.path.join(root,x) for root, _, files in os.walk("aclImdb/train/pos/", topdown=False) for x in files if x.endswith(".txt")]

In [0]:
# Return texts from path
# if the text is too short, then it will omit it. 
def fetch_text(path):
  with open(path) as f:
    text = f.read()
    if len(text.split()) < 50:
      return False
    else:
      return text

In [0]:
# Next, the files are retrieved as the list of texts.
neg_test = list(filter(None, [fetch_text(x) for x in neg_test]))
pos_test = list(filter(None, [fetch_text(x) for x in pos_test]))
neg_train = list(filter(None, [fetch_text(x) for x in neg_train]))
pos_train = list(filter(None, [fetch_text(x) for x in pos_train]))

In [0]:
# Aggregate all lists into one:

all_text = neg_test + pos_test + neg_train + pos_train

In [0]:
# Pre-process these given texts, by removing stop words and stemming.
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_short
stopwords = gensim.parsing.preprocessing.STOPWORDS
all_text_processed = [preprocess_string(x.lower(), filters = [strip_tags, strip_short, strip_punctuation, strip_multiple_whitespaces]) for x in all_text]

In [0]:
# Display how one of them is pre-processed:

all_text_processed = [[x for x in str_list if len(x) > 1 and not x in stopwords] for str_list in all_text_processed]

In [94]:
print(all_text_processed[0])

['casper', 'van', 'dien', 'michael', 'rooker', 'generally', 'relegated', 'movies', 'movie', 'fails', 'convey', 'slightest', 'sense', 'excitement', 'fear', 'dread', 'count', 'dread', 'sitting', 'rest', 'garbage', 'direction', 'amateurish', 'annoying', 'cuts', 'jerky', 'movement', 'hides', 'fact', 'killer', 'near', 'victims', 'attacks', 'killer', 'cheap', 'skull', 'mask', 'black', 'hood', 'liked', 'better', 'fighting', 'man', 'laziest', 'jobs', 'character', 'design', 've', 'seen', 'mean', 'skeletor', 'horse', 'supposed', 'scary', 'supernatural', 'creature', 'supposed', 'seriously', 'scenes', 'dude', 'riding', 'woods', 'horse', 'barely', 'stay', 'interspersed', 'scenes', 'soldiers', 'shooting', 'randomly', 'woods', 'thinking', 'shoot', 'ghost', 'occasionally', 'skeletor', 'shoot', 'arrow', 'ride', 'stab', 'revealing', 'corny', 'effects', 'generally', 'enjoy', 'sci', 'channel', 'fare', 'basic', 'cheese', 'level', 'film', 'inept', 'level', 'enjoyment', 'dolph', 'lundgren', 'need']


In [0]:
# Create dictionary & BOW:
dictionary = Dictionary(all_text_processed)
corpus = [dictionary.doc2bow(x) for x in all_text_processed]

In [0]:
lda_imdb = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)

In [97]:
# You can see that the topic is all about movie and drama series.

lda_imdb.show_topics()

[(0,
  '0.013*"movie" + 0.011*"like" + 0.007*"good" + 0.006*"time" + 0.006*"funny" + 0.005*"series" + 0.004*"great" + 0.004*"comedy" + 0.004*"episode" + 0.003*"best"'),
 (1,
  '0.036*"movie" + 0.011*"like" + 0.008*"people" + 0.007*"good" + 0.007*"film" + 0.006*"horror" + 0.006*"movies" + 0.006*"bad" + 0.004*"think" + 0.004*"story"'),
 (2,
  '0.011*"film" + 0.004*"films" + 0.004*"time" + 0.003*"like" + 0.003*"best" + 0.003*"movie" + 0.003*"good" + 0.003*"years" + 0.003*"way" + 0.002*"people"'),
 (3,
  '0.015*"film" + 0.005*"story" + 0.005*"life" + 0.004*"man" + 0.004*"love" + 0.004*"young" + 0.003*"like" + 0.003*"time" + 0.003*"character" + 0.003*"great"'),
 (4,
  '0.030*"film" + 0.028*"movie" + 0.011*"good" + 0.009*"like" + 0.008*"great" + 0.008*"story" + 0.007*"time" + 0.005*"think" + 0.005*"watch" + 0.005*"seen"')]

In [99]:
dictionary

In [0]:
from collections import Counter

# Count numbers of words in corpus.
def count_words_in_corpus(corpus):
  count_dict = Counter() 
  for word_count in corpus:
    for key, value in word_count:
      count_dict.update({dictionary[key] : value})

  return count_dict

In [0]:
count_of_words = count_words_in_corpus(corpus)

In [0]:
sorted_count = sorted(count_of_words.items(), key = lambda x: x[1], reverse = True)

In [124]:
print("\n".join([str(x) for x in sorted_count[:20]]))

('movie', 87076)
('film', 79252)
('like', 39981)
('good', 29489)
('time', 24923)
('story', 22941)
('bad', 18248)
('people', 18109)
('great', 17878)
('way', 15592)
('movies', 15154)
('characters', 14360)
('think', 14270)
('character', 13859)
('watch', 13807)
('films', 13699)
('seen', 13231)
('love', 12913)
('life', 12867)
('plot', 12851)


In [0]:
# References
# Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. 
# The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).